[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/chanwkimlab/MarcoPolo/blob/main/notebooks/tutorial.ipynb)

# Setup

**Start the colab kernel with GPU**: Runtime -> Change runtime type -> GPU

## Install dependencies

In [ ]:
!pip install marcopolo-pytorch --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 614 kB 4.8 MB/s 
     |████████████████████████████████| 96 kB 2.8 MB/s 
     |████████████████████████████████| 38.1 MB 2.5 MB/s 
     |████████████████████████████████| 11.2 MB 52.3 MB/s 
     |████████████████████████████████| 2.0 MB 52.2 MB/s 
     |████████████████████████████████| 930 kB 50.4 MB/s 
     |████████████████████████████████| 88 kB 9.5 MB/s 
     |████████████████████████████████| 1.1 MB 58.0 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=3e71036eba287a07a3567bb27056e630f811cf53267c215c5a28baa7bc49a4e7
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=5c2c96f90bc47e5f97c2028ee

In [ ]:
!pip install matplotlib==3.1.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.1 MB 4.3 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.2
    Uninstalling matplotlib-3.5.2:
      Successfully uninstalled matplotlib-3.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scanpy 1.9.1 requires matplotlib>=3.4, but you have matplotlib 3.1.3 which is incompatible.
marcopolo-pytorch 1.0.9 requires matplotlib>=3.3.0, but you have matplotlib 3.1.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


# Run MarcoPolo

## Import packages

In [ ]:
# Import packages
import pickle

import numpy as np
import pandas as pd
import torch
import anndata as ad
import scanpy as sc
import matplotlib.pyplot as plt

import MarcoPolo

assert torch.cuda.is_available(), "Make sure that you started the colab kernel with GPU: Runtime -> Change runtime type -> GPU"

## Read scRNA-seq data

You can use **example data** or **your own data**.

It should be in a AnnData format. `.X` should contain a raw count matrix of shape (# cells, # genes). You can explore example datasets below

### example data
We have prepared two example data: the human embryogenic stem cell (hESC) dataset of Koh et al. and the liver dataset of MacParland et al.    

In [ ]:
!wget https://raw.githubusercontent.com/chanwkimlab/MarcoPolo/main/notebooks/example/hESC.h5ad
!wget https://raw.githubusercontent.com/chanwkimlab/MarcoPolo/main/notebooks/example/HumanLiver.h5ad
    
anndata_path = "hESC.h5ad"

# Read anndata. `anndata_path` should be in a `h5ad` format.
adata = ad.read(anndata_path)

# For fast debugging, only test first 1,000 genes.
adata = adata[:, :1000]

--2022-06-17 17:24:04--  https://raw.githubusercontent.com/chanwkimlab/MarcoPolo/main/notebooks/example/hESC.h5ad
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20842419 (20M) [application/octet-stream]
Saving to: ‘hESC.h5ad’

hESC.h5ad           100%[===================>]  19.88M  --.-KB/s    in 0.1s    

2022-06-17 17:24:05 (149 MB/s) - ‘hESC.h5ad’ saved [20842419/20842419]

--2022-06-17 17:24:05--  https://raw.githubusercontent.com/chanwkimlab/MarcoPolo/main/notebooks/example/HumanLiver.h5ad
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 20

### your own data
You can upload your own AnnData single cell file to this session. If you intend to use the example data, please run the following cell and upload your data.

In [ ]:
from google.colab import files
uploaded = files.upload()

for file_name in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=file_name, length=len(uploaded[file_name])))
    
anndata_path = file_name

# Read anndata. `anndata_path` should be in a `h5ad` format.
adata = ad.read(anndata_path)

## (1) Run regression

In [ ]:
# (1) Run regression
# Calculate size factor
if "size_factor" not in adata.obs.columns:
    norm_factor = sc.pp.normalize_total(adata, exclude_highly_expressed=True, max_fraction= 0.2, inplace=False)["norm_factor"]
    adata.obs["size_factor"] = norm_factor/norm_factor.mean()
    print("size factor was calculated")
regression_result = MarcoPolo.run_regression(adata=adata, size_factor_key="size_factor",
                         num_threads=2, device="cuda:0")
# If you use a local machine, you can set `num_threads` to higher than 1 (maybe upto 4), which will speed up the regression a lot. For some reason, num_threads>1 does not work on colab.

with open(f"{anndata_path}.regression_result.pickle", "wb") as f:
    pickle.dump(regression_result, f)

<INFO> Currently, you are using 2 threads for regression. If you encounter any memory issues, try to set `num_threads` to 1.
The numbers of clusters to test: [1, 2]
Y: (446, 1000) X: (446, 1) s: (446,)


## (2) Find markers

In [ ]:
# (2) Find markers
markers_result = MarcoPolo.find_markers(adata=adata, regression_result=regression_result)
with open(f"{anndata_path}.markers_result.pickle", "wb") as f:
    pickle.dump(markers_result, f)

Assign cells to on-cells and off-cells...
Calculating voting score...
Calculating proximity score...
Calculating bimodality score...
Calculating MarcoPolo score...


## (3) Generate report

In [ ]:
# Obtain tSNE coordinates if it does not exist in the adata.
if "X_tsne" not in adata.obsm.keys():
    sc.tl.tsne(adata=adata)

In [ ]:
# (3) Generate report
MarcoPolo.generate_report(adata=adata, size_factor_key="size_factor", 
                          regression_result=regression_result, 
                          gene_scores=markers_result, 
                          output_dir="./",  
                          low_dim_key="X_tsne",
                          cell_color_key="cell_type",
                          gene_info_path="https://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz",
                          top_num_html=1000,
                          top_num_image=1000)

Assign cells to on-cells and off-cells...
Annotating genes with the gene info: https://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz


100%|██████████| 1000/1000 [00:20<00:00, 48.02it/s, Num. of unmatched genes=47]


47 not matched genes: ENSG00000198804, ENSG00000210082, ENSG00000198712, ENSG00000198938, ENSG00000198727, ENSG00000211459, ENSG00000198899, ENSG00000198886, ENSG00000198763, ENSG00000198888, ENSG00000198786, ENSG00000212907, ENSG00000225840, ENSG00000274474, ENSG00000183311, ENSG00000223367, ENSG00000226225, ENSG00000235650, ENSG00000096150, ENSG00000273673, ...
Generating table files...


findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans.


Generating image files...
Drawing figures
size factor corrected


  0%|          | 0/1000 [00:00<?, ?it/s]findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans.
100%|██████████| 1000/1000 [12:56<00:00,  1.29it/s]


# Download report

## compress the report folder 

In [ ]:
!tar -zcf report.tar.gz report

## trigger download

In [ ]:
from google.colab import files

files.download('report.tar.gz')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>